# Importing necessary Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Standard plotly imports
#import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
#import cufflinks
#import cufflinks as cf
import plotly.figure_factory as ff

# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
#cufflinks.go_offline(connected=True)

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from xgboost import XGBClassifier
import xgboost as xgb

## Hyperopt modules
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from functools import partial

import os
import gc
print(os.listdir("./data"))

['sample_submission.csv', 'test_identity.csv', 'test_transaction.csv', 'train_identity.csv', 'train_transaction.csv']


# Importing train Datasets

In [5]:
# train data
df_id = pd.read_csv('./data/train_identity.csv')
df_trans = pd.read_csv('./data/train_transaction.csv')

MemoryError: 

!pip install plotly

!pip install xgboost

## function- resumeTable(df)

In [2]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary

## function - reduce_mem_usage
function to reduce the DF size

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## function - CalcOutliers(df_num)

In [4]:
def CalcOutliers(df_num):
    # 평균과 표준편차를 구한다.
    data_mean , data_std = np.mean(df_num), np.std(df_num)
    
    # 높은곳과 낮은곳 둘다를 자르는 라인을 설정
    cut = data_std *3 #표준편차 *3
    
    # higher, lower 컷값을 계산한다.
    lower, upper = data_mean - cut, data+cut
    
    # creating an array of lower, higher and total outlier values
    outliers_lower=[x for x in df_num if x<lower]
    outliers_higher=[x for x in df_num if x>upper]
    outlier_total = [x for x in df_num if x<lower or x>upper]
    
    # array without outlier values
    outliers_removed = [ x for x in df_num if x>lower and x<upper]
    
    # printing total number of values in lower cut of outliers
    print('Identified lowest outliers: %d' %len(outliers_lower))
    
    # printing total number of values in  higher cut of outliers 
    print('Identified upper outliers: %d' %len(outliers_higher))
    
    # printing total number of values outliers of both sides(lower, higher)
    print('Total outlier observations: %d' %len(outliers_total))
    
    # prining total number of non-outliers values
    print('Total percentual of Outliers: ', round((len(outliers_total)/len(outlers_removed))*100, 4))
    
    return

# reducing memory

In [ ]:
df_trans = reduce_mem_usage(df_trans)
df_id = reduce_mem_usage(df_id)

# Knowing the data

In [ ]:
resumetable(df_trans)[:25]